In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# Paramètres
Re = 100.0  # Reynolds
Ra = 1000.0 # Rayleigh
Pr = 0.71   # Air
Lx = 1.0     # Cavité carrée
Ly = 1.0
nx, ny = 40, 40 # Maillage
dx = Lx / nx
dy = Ly / ny
dt = 0.001

# Initialisation des champs
T = np.zeros((nx, ny))
psi = np.zeros((nx, ny))
omega = np.zeros((nx, ny))

In [ ]:
def thomas_algorithm(a, b, c, d):
    """
    Résout le système Ax = d où A est tridiagonale.
    a, b, c, d sont des tableaux numpy de taille n.
    Note : a[0] et c[-1] sont ignorés.
    Résout ax_{i-1} + bx_i + cx_{i+1} = d_i
    """
    n = len(d)
    # On crée des copies pour ne pas modifier les tableaux d'origine
    cp = np.zeros(n)
    dp = np.zeros(n)
    sol = np.zeros(n)

    # Phase de descente (Forward elimination)
    cp[0] = c[0] / b[0]
    dp[0] = d[0] / b[0]
    
    for i in range(1, n):
        m = b[i] - a[i] * cp[i-1]
        cp[i] = c[i] / m
        dp[i] = (d[i] - a[i] * dp[i-1]) / m

    # Phase de remontée (Back substitution)
    sol[-1] = dp[-1]
    for i in range(n-2, -1, -1):
        sol[i] = dp[i] - cp[i] * sol[i+1]
        
    return sol

def solve_ADI_energy(T, psi, Re, Pr, dt, dx, dy):
    nx, ny = T.shape
    alpha = 1.0 / (Re * Pr)
    T_new = T.copy()
    
    # Calcul des vitesses à partir de psi
    u = np.zeros_like(psi)
    v = np.zeros_like(psi)
    u[1:-1, 1:-1] = (psi[1:-1, 2:] - psi[1:-1, :-2]) / (2 * dy)
    v[1:-1, 1:-1] = -(psi[2:, 1:-1] - psi[:-2, 1:-1]) / (2 * dx)

    # Étape 1 : Implicite en X, Explicite en Y (demi-pas dt/2)
    for j in range(1, ny - 1):
        a = np.full(nx-2, -alpha*dt/(2*dx**2) - u[1:-1, j]*dt/(4*dx))
        b = np.full(nx-2, 1 + alpha*dt/(dx**2))
        c = np.full(nx-2, -alpha*dt/(2*dx**2) + u[1:-1, j]*dt/(4*dx))
        
        # Termes explicites en Y
        d = T[1:-1, j] + (alpha*dt/(2*dy**2)) * (T[1:-1, j+1] - 2*T[1:-1, j] + T[1:-1, j-1]) - \
            (v[1:-1, j]*dt/(4*dy)) * (T[1:-1, j+1] - T[1:-1, j-1])
        
        # Conditions aux limites (Dirichlet sur les murs gauche/droite)
        d[0] -= a[0] * T_new[0, j]
        d[-1] -= c[-1] * T_new[-1, j]
        T_new[1:-1, j] = thomas_algorithm(a, b, c, d)

    # Étape 2 : Explicite en X, Implicite en Y (demi-pas dt/2)
    # (Logique similaire en inversant les rôles de X et Y)
    return T_new

